<h2 style="text-align: center;">User|Customers Interface</h2>

- Enter phase of the user now that  we are done with admin
- Some of the functionality of the user
    - Sign up
        - Name
        - email
        - Password
    - Creating a profile
        - Location
        - Mobile Phone
        - Date of birth
        - Profile picture
        - Invitation code
        -  Consent for sending marketing news
        - Data Protection
    - Log in 
        - User Name
        - Password
        - Reset Password
    - View 
        - Fileter products
        - order 
    - Place order    
        - Fill their Cart
        - Pay:
            - Mpesa
            - card
            - Paypall
        - Delivery
    - After Sales service
        - Write review
        - Return products

Great—you’ve laid out the user‐facing side beautifully. Let’s turn that into a phased plan so we can tackle it piece by piece. Once we agree on the plan, we’ll jump into defining the data models, forms, routes, and templates.

---

## 📋 Phase 0: Project Structure & Blueprints

1. **Create a `users` blueprint**

   * `app/users/__init__.py`
   * `app/users/models.py`
   * `app/users/forms.py`
   * `app/users/routes.py`
   * `app/users/templates/…`

2. **Configure the blueprint in `app.py`**

   ```python
   from users import users_bp
   app.register_blueprint(users_bp, url_prefix='/user')
   ```

---

## 🛠 Phase 1: Authentication & Profile

1. **Data models** (`users/models.py`)

   ```python
   class User(db.Model, UserMixin):
       id             = db.Column(db.Integer, primary_key=True)
       username       = db.Column(db.String(30), unique=True, nullable=False)
       email          = db.Column(db.String(120), unique=True, nullable=False)
       password_hash  = db.Column(db.String(128), nullable=False)
       # profile fields
       phone          = db.Column(db.String(20))
       location       = db.Column(db.String(200))
       date_of_birth  = db.Column(db.Date)
       profile_pic    = db.Column(db.String(200))  # path/URL
       invite_code    = db.Column(db.String(20), unique=True)
       marketing_opt_in = db.Column(db.Boolean, default=False)
       created_at     = db.Column(db.DateTime, default=db.func.current_timestamp())
   ```

2. **Forms** (`users/forms.py`)

   * `RegistrationForm`—username, email, password, confirm, consent checkbox
   * `LoginForm`—username/email & password
   * `ProfileForm`—phone, location, dob, file upload, invite code, marketing opt‑in

3. **Routes** (`users/routes.py`)

   * `GET, POST /register` → `RegistrationForm`
   * `GET, POST /login` → `LoginForm`
   * `GET /logout`
   * `GET, POST /profile` → require login, show/edit profile

4. **Templates**

   * `register.html`, `login.html`, `profile.html`

---

## 🛍 Phase 2: Product Browsing & Cart

1. **Models**

   * `Cart` (one per user, one‑to‑many `CartItem`)
   * `CartItem` (`user_id`, `item_id`, `quantity`)

2. **Forms**

   * `AddToCartForm` (hidden item\_id + quantity)
   * `CartUpdateForm` (update quantities/remove)

3. **Routes**

   * `GET /products` → filter & pagination
   * `GET /product/<id>` → product detail + add‑to‑cart
   * `GET /cart` → view & update cart
   * `POST /cart/add` → add item
   * `POST /cart/update` → update/remove

4. **Templates**

   * `products.html` (list + filters)
   * `product_detail.html`
   * `cart.html`

---

## 💳 Phase 3: Checkout & Payment

1. **Models**

   * `Order` (`user_id`, `status`, `total_amount`, `created_at`)
   * `OrderItem` (`order_id`, `item_id`, `quantity`, `unit_price`)

2. **Forms**

   * `CheckoutForm` (delivery address, payment method choice)

3. **Routes**

   * `GET, POST /checkout` → show summary, collect payment info
   * `POST /payment/mpesa` (or redirect to external gateway)
   * webhook endpoints for payment status

4. **Templates**

   * `checkout.html`
   * `order_confirmation.html`

---

## 📝 Phase 4: After‑Sales (Reviews & Returns)

1. **Models**

   * `Review` (`user_id`, `item_id`, `rating`, `comment`, `created_at`)
   * `ReturnRequest` (`order_item_id`, `reason`, `status`, `created_at`)

2. **Forms**

   * `ReviewForm`
   * `ReturnForm`

3. **Routes**

   * `POST /review/<item_id>` → submit review
   * `GET, POST /return/<order_item_id>` → request return

4. **Templates**

   * Integrated into `order_history.html`

---

## ✅ Next Steps

1. **Spin up the `users` blueprint** and scaffold the files.
2. **Implement the `User` model** and database migration.
3. **Build and test the registration flow** end‑to‑end (form → model → login).

Shall we start with Phase 1’s **User model**, **migration**, and **RegistrationForm**? Let me know, and I’ll drop in the exact code snippets!


##  Image URL scaraber 


In [ ]:
import os
import requests


In [9]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file
access_key = os.getenv("UNSPLASH_ACCESS_KEY")
secret_key = os.getenv("UNSPLASH_SECRET_KEY")
item_name = "laptop"  # Replace with the actual item name or variable

headers = {
    'Accept-Version': 'v1',
    'Authorization': f'Client-ID {access_key}'
}

params = {
    'query': item_name,
    'orientation': 'squarish',
}
response = requests.get('https://api.unsplash.com/photos/random', headers=headers, params=params)

In [11]:
print(response.raise_for_status) # Raise an error for bad responses

<bound method Response.raise_for_status of <Response [200]>>


In [13]:
import os
import requests
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env file
# Load from environment variables (recommended)
access_key = os.getenv("UNSPLASH_ACCESS_KEY")
item_name = "samsung phone"  # Replace this with any test item

# Check if key is available
if not access_key:
    raise EnvironmentError("UNSPLASH_ACCESS_KEY is not set in environment variables.")

headers = {
    'Accept-Version': 'v1',
    'Authorization': f'Client-ID {access_key}'
}

params = {
    'query': item_name,
    'orientation': 'squarish'
}

# Make request
response = requests.get('https://api.unsplash.com/photos/random', headers=headers, params=params)

# Check the response
if response.status_code == 200:
    data = response.json()
    image_url = data.get("urls", {}).get("regular")
    if image_url:
        print(f"✅ Image found for '{item_name}':\n{image_url}")
    else:
        print("⚠️ Image URL not found in response.")
else:
    print(f"❌ API call failed with status {response.status_code}")
    print(response.text)


✅ Image found for 'samsung phone':
https://images.unsplash.com/photo-1651466122871-de0527f8b9a1?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w3ODM3NzV8MHwxfHJhbmRvbXx8fHx8fHx8fDE3NTM2NDgyODZ8&ixlib=rb-4.1.0&q=80&w=1080
